In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os
import shutil
import numpy as np
import random

In [2]:
def create_validation_split(train_dir, val_dir, val_split=0.2):
    if not os.path.exists(val_dir):
        os.makedirs(val_dir)

    class_names = os.listdir(train_dir)

    for class_name in class_names:
        class_train_dir = os.path.join(train_dir, class_name)
        class_val_dir = os.path.join(val_dir, class_name)

        if not os.path.exists(class_val_dir):
            os.makedirs(class_val_dir)

        images = os.listdir(class_train_dir)
        num_val_images = int(len(images) * val_split)
        val_images = random.sample(images, num_val_images)

        for image in val_images:
            src_path = os.path.join(class_train_dir, image)
            dst_path = os.path.join(class_val_dir, image)
            shutil.move(src_path, dst_path)
        print(f"Moved {num_val_images} images from {class_train_dir} to {class_val_dir}")

# Example usage:
train_dir = 'train'
val_dir = 'validation'
create_validation_split(train_dir, val_dir, val_split=0.2)

Moved 52 images from train\Black-grass to validation\Black-grass
Moved 78 images from train\Charlock to validation\Charlock
Moved 57 images from train\Cleavers to validation\Cleavers
Moved 122 images from train\Common Chickweed to validation\Common Chickweed
Moved 44 images from train\Common wheat to validation\Common wheat
Moved 95 images from train\Fat Hen to validation\Fat Hen
Moved 130 images from train\Loose Silky-bent to validation\Loose Silky-bent
Moved 44 images from train\Maize to validation\Maize
Moved 103 images from train\Scentless Mayweed to validation\Scentless Mayweed
Moved 46 images from train\Shepherds Purse to validation\Shepherds Purse
Moved 99 images from train\Small-flowered Cranesbill to validation\Small-flowered Cranesbill
Moved 77 images from train\Sugar beet to validation\Sugar beet


In [2]:
training_set = tf.keras.utils.image_dataset_from_directory(
    'train',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)

# Create validation dataset
validation_set = tf.keras.utils.image_dataset_from_directory(
    'validation',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=False,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False
)

# Get the number of classes
class_names = training_set.class_names
num_classes = len(class_names)

Found 3805 files belonging to 12 classes.
Found 947 files belonging to 12 classes.


In [3]:
# Data augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal_and_vertical'),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2)
])

# Normalize the data
normalization_layer = layers.Rescaling(1./255)

# Apply data augmentation and normalization
training_set = training_set.map(lambda x, y: (data_augmentation(normalization_layer(x)), y))
validation_set = validation_set.map(lambda x, y: (normalization_layer(x), y))

In [4]:
# Build the model
def build_model(input_shape, num_classes):
    model = models.Sequential([
        layers.Input(shape=input_shape),
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Define input shape
img_height, img_width = 128, 128
input_shape = (img_height, img_width, 3)

# Create an instance of the model
model = build_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()

# Train the model
history = model.fit(
    training_set,
    validation_data=validation_set,
    epochs=20
)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 128)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 12)                  │           1,548 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 111,308 (434.80 KB)

 Trainable params: 111,308 (434.80 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 59s 476ms/step - accuracy: 0.1410 - loss: 2.4198 - val_accuracy: 0.2408 - val_loss: 2.1711
Epoch 2/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 55s 458ms/step - accuracy: 0.2727 - loss: 2.0200 - val_accuracy: 0.3200 - val_loss: 1.8547
Epoch 3/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 57s 473ms/step - accuracy: 0.3169 - loss: 1.8535 - val_accuracy: 0.3495 - val_loss: 1.7408
Epoch 4/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 56s 461ms/step - accuracy: 0.3451 - loss: 1.7407 - val_accuracy: 0.4013 - val_loss: 1.6115
Epoch 5/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 55s 459ms/step - accuracy: 0.4205 - loss: 1.6106 - val_accuracy: 0.4615 - val_loss: 1.5043
Epoch 6/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 55s 461ms/step - accuracy: 0.4527 - loss: 1.5238 - val_accuracy: 0.4952 - val_loss: 1.4170
Epoch 7/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 55s 460ms/step - accuracy: 0.4730 - loss: 1.4341 - val_accuracy: 0.4604 - val_loss: 1.4104
Epoch 8/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 56s 467ms/step - accuracy: 0.4796 - loss: 1

In [5]:
#Training set Accuracy
train_loss, train_acc = model.evaluate(training_set)
print('Training accuracy:', train_acc*100)

119/119 ━━━━━━━━━━━━━━━━━━━━ 28s 232ms/step - accuracy: 0.6533 - loss: 1.0064
Training accuracy: 66.07096195220947


In [6]:
#Validation set Accuracy
val_loss, val_acc = model.evaluate(validation_set)
print('Validation accuracy:', val_acc*100)

30/30 ━━━━━━━━━━━━━━━━━━━━ 5s 158ms/step - accuracy: 0.7001 - loss: 0.8754
Validation accuracy: 69.16578412055969
